<h1> Facility Location Problem </h1>

In [173]:
from IPython.display import IFrame
IFrame("./FacilityLocationProblem_2019.pdf", width=1000, height=600)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mathematical-Formulation" data-toc-modified-id="Mathematical-Formulation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Mathematical Formulation</a></span></li><li><span><a href="#Install-Required-Packages-and-define-Instance" data-toc-modified-id="Install-Required-Packages-and-define-Instance-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Install Required Packages and define Instance</a></span><ul class="toc-item"><li><span><a href="#Instance-Specifics" data-toc-modified-id="Instance-Specifics-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Instance Specifics</a></span></li></ul></li><li><span><a href="#Set-up-Model" data-toc-modified-id="Set-up-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set up Model</a></span></li><li><span><a href="#Set-up-Decision-Variables" data-toc-modified-id="Set-up-Decision-Variables-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Set up Decision Variables</a></span></li><li><span><a href="#Express-The-Constraints" data-toc-modified-id="Express-The-Constraints-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Express The Constraints</a></span><ul class="toc-item"><li><span><a href="#Not-more-than-N_DC-DC's" data-toc-modified-id="Not-more-than-N_DC-DC's-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Not more than N_DC DC's</a></span></li><li><span><a href="#Satisfy-Demand,-and-Flow-Conservation" data-toc-modified-id="Satisfy-Demand,-and-Flow-Conservation-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Satisfy Demand, and Flow Conservation</a></span></li><li><span><a href="#Satisfy-Production-Capacity" data-toc-modified-id="Satisfy-Production-Capacity-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Satisfy Production Capacity</a></span></li></ul></li><li><span><a href="#Define-The-Objective-Function" data-toc-modified-id="Define-The-Objective-Function-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Define The Objective Function</a></span></li><li><span><a href="#Solve" data-toc-modified-id="Solve-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Solve</a></span></li><li><span><a href="#Further-Analyse-of-the-Solution" data-toc-modified-id="Further-Analyse-of-the-Solution-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Further Analyse of the Solution</a></span></li><li><span><a href="#Render-A-Visualization" data-toc-modified-id="Render-A-Visualization-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Render A Visualization</a></span></li><li><span><a href="#Download-the-code" data-toc-modified-id="Download-the-code-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Download the code</a></span></li></ul></div>

## Mathematical Formulation

min $\sum_{i = 1}^n \sum_{j = 1}^n c_{ij}X_{SHIP, ij} + \sum_{i=1}^n f_i X_{DC, i}$

st.

Not more than $N_{DC}$ locations: $\sum_{i = 1}^n X_{DC, i} <= N_{DC}$ 

Satisfy the demand-constraint: $\sum_{i=1}^m X_{SHIP, ij} = D[j] \qquad \qquad\qquad \forall j$

Non-negative shipping: $X_{SHIP, ij} >= 0\qquad \qquad \qquad \quad \qquad\qquad \forall i,j$

No cheating: $X_{SHIP, ii} = 0 \qquad \qquad \qquad  \qquad \qquad \qquad\qquad \quad \forall i$

Satisfy the Supply-constraint: $ S[i]*X_{DC, i} >= \sum_{j = 1}^m X_{SHIP, i,j} \quad \quad \quad \forall i$

## Install Required Packages and define Instance

In [174]:
import pandas as pd
import numpy as np
import sys

try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')      

from docplex.mp.environment import Environment
from docplex.mp.model import Model
#env = Environment()
#env.print_information()

In [175]:
url = open("URL").read()
key = open("APIKEY").read()

### Instance Specifics

In [185]:
c = pd.read_csv("./cities_v2.csv", sep = ",").dropna()
c.index = c['Unnamed: 0']
c.drop_duplicates(keep=False,inplace=True) 
cities = list(c.columns)
N_DC = 37

D = [100 for i in range(len(cities))]
F = [150000 for i in range(len(cities))]
S = [4000 for i in range(len(cities))]

model_name = "Facility Location Problem"
cloud = False

In [186]:
c.shape

(500, 501)

In [177]:
c

,Malishevë,Prizren,Zubin Potok,Kamenicë,Viti,Shtërpcë,Shtime,Vushtrri,Dragash,Podujevë,...,Viļāni,Varakļāni,Ape,Madona,Jaunpiebalga,Lubāna,Gulbene,Balvi,Aknīste,Viesīte
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Malishevë,0.000000,29.847177,48.291776,69.228460,53.392911,35.556495,24.728844,42.032424,51.256310,59.949238,...,1625.672207,1628.542009,1726.317279,1644.781165,1676.678996,1659.136905,1688.380555,1693.229478,1563.142717,1579.984912
Prizren,29.847177,0.000000,78.024089,80.055205,52.302771,23.849135,34.686324,70.287644,22.029629,85.873099,...,1654.808026,1657.725735,1755.621913,1674.106374,1706.071618,1688.365991,1717.633681,1722.367735,1592.492638,1609.390685
Zubin Potok,48.291776,78.024089,0.000000,81.800518,85.705960,80.007194,60.706053,24.819466,98.806110,40.749773,...,1580.099335,1582.847362,1680.302949,1598.716622,1630.430625,1613.323768,1642.503345,1647.645871,1517.016612,1533.705395
Kamenicë,69.228460,80.055205,81.800518,0.000000,33.870802,59.013881,47.171869,57.025152,97.876592,48.876304,...,1600.140882,1603.501294,1702.541895,1621.305751,1654.011503,1634.558224,1664.045437,1667.675238,1540.029464,1557.561820
Viti,53.392911,52.302771,85.705960,33.870802,0.000000,28.733756,28.977401,64.333429,66.733296,67.024938,...,1631.872021,1635.141896,1733.952989,1652.648269,1685.197286,1666.115611,1695.559970,1699.418489,1571.289197,1588.685044
Shtërpcë,35.556495,23.849135,80.007194,59.013881,28.733756,0.000000,21.569719,65.108994,38.884412,75.892244,...,1646.677862,1649.763052,1748.102718,1666.678840,1698.923463,1680.564639,1709.919015,1714.238058,1585.176546,1602.311755
Shtime,24.728844,34.686324,60.706053,47.171869,28.977401,21.569719,0.000000,43.780200,55.300608,54.568336,...,1625.446059,1628.503269,1726.775962,1645.338211,1677.545909,1659.279449,1688.620791,1693.007040,1563.819701,1580.924474
Vushtrri,42.032424,70.287644,24.819466,57.025152,64.333429,65.108994,43.780200,0.000000,92.284600,20.604382,...,1584.562728,1587.501043,1685.472305,1603.974284,1636.004184,1618.165276,1647.447561,1652.123097,1522.383412,1539.339013
Dragash,51.256310,22.029629,98.806110,97.876592,66.733296,38.884412,55.300608,92.284600,0.000000,107.794246,...,1676.736668,1679.638167,1777.484783,1695.959210,1727.888399,1710.261409,1739.519362,1744.295707,1614.332794,1631.199597


In [178]:
c.shape

(500, 499)

## Set up Model

In [179]:
mdl = Model(model_name)

In [180]:
if cloud:
    from docplex.mp.context import Context
    context = Context.make_default_context()

    context.solver.docloud.url = url
    context.solver.docloud.key = key
    context.solver.agent = 'docloud'
else:
    from docplex.cp.config import context
    context.solver.agent = 'local'
    file = r"C:\Program Files\IBM\ILOG\CPLEX_Studio_Community128\cpoptimizer\bin\x64_win64\cpoptimizer.exe"
    context.solver.local.execfile = file

## Set up Decision Variables

In [181]:
X_DC = mdl.binary_var_dict(cities, name="is_DC__")
X_ship = mdl.integer_var_matrix(c, cities, name = "IsActive__")
X_prod = mdl.integer_var_dict(cities,  name = "prod___")

In [182]:
X_prod

{'Malishevë': docplex.mp.Var(type=I,name='prod____Malishevë'),
 'Prizren': docplex.mp.Var(type=I,name='prod____Prizren'),
 'Zubin Potok': docplex.mp.Var(type=I,name='prod____Zubin Potok'),
 'Kamenicë': docplex.mp.Var(type=I,name='prod____Kamenicë'),
 'Viti': docplex.mp.Var(type=I,name='prod____Viti'),
 'Shtërpcë': docplex.mp.Var(type=I,name='prod____Shtërpcë'),
 'Shtime': docplex.mp.Var(type=I,name='prod____Shtime'),
 'Vushtrri': docplex.mp.Var(type=I,name='prod____Vushtrri'),
 'Dragash': docplex.mp.Var(type=I,name='prod____Dragash'),
 'Podujevë': docplex.mp.Var(type=I,name='prod____Podujevë'),
 'Fushë Kosovë': docplex.mp.Var(type=I,name='prod____Fushë Kosovë'),
 'Kaçanik': docplex.mp.Var(type=I,name='prod____Kaçanik'),
 'Klinë': docplex.mp.Var(type=I,name='prod____Klinë'),
 'Leposaviq': docplex.mp.Var(type=I,name='prod____Leposaviq'),
 'Pejë': docplex.mp.Var(type=I,name='prod____Pejë'),
 'Rahovec': docplex.mp.Var(type=I,name='prod____Rahovec'),
 'Gjilan': docplex.mp.Var(type=I,name='p

## Express The Constraints

### Not more than N_DC DC's

In [183]:
mdl.add_constraint(mdl.sum(X_DC) <= N_DC)

docplex.mp.LinearConstraint[](is_DC___Malishevë+is_DC___Prizren+is_DC___Zubin Potok+is_DC___Kamenicë+is_DC___Viti+is_DC___Shtërpcë+is_DC___Shtime+is_DC___Vushtrri+is_DC___Dragash+is_DC___Podujevë+is_DC___Fushë Kosovë+is_DC___Kaçanik+is_DC___Klinë+is_DC___Leposaviq+is_DC___Pejë+is_DC___Rahovec+is_DC___Gjilan+is_DC___Lipjan+is_DC___Obiliq+is_DC___Gjakovë+is_DC___Pristina+is_DC___Deçan+is_DC___Istog+is_DC___Hani i Elezit+is_DC___Junik+is_DC___Kllokot+is_DC___Mamushë+is_DC___Partesh+is_DC___Ranillug+is_DC___Ferizaj+is_DC___Zveçan+is_DC___Suharekë+is_DC___Gllogovc+is_DC___Mitrovicë+is_DC___Skenderaj+is_DC___Novobërdë+is_DC___Graçanicë+is_DC___Longyearbyen+is_DC___Sanaa+is_DC___Marib+is_DC___Al Jabīn+is_DC___Hajjah+is_DC___Ibb+is_DC___Al Hudaydah+is_DC___Lahij+is_DC___Al Maḩwīt+is_DC___Taizz+is_DC___Sadah+is_DC___Dhamar+is_DC___‘Amrān+is_DC___Aḑ Ḑāli‘+is_DC___Saywun+is_DC___Al Bayda+is_DC___Zinjibār+is_DC___Rida+is_DC___'Ataq+is_DC___Al Ghaydah+is_DC___Sayhut+is_DC___Al Mukalla+is_DC___Al Ḩa

### Satisfy Demand, and Flow Conservation

In [184]:
X_ship[('Middelburg.1', 'Malishevë')]

KeyError: ('Middelburg.1', 'Malishevë')

In [ ]:
for i in range(len(cities)):
    for j in range(len(cities)):
        mdl.add_constraint(X_ship[(cities[i],cities[j])] >= 0)
    mdl.add_constraint(D[i]
                       == mdl.sum(X_ship[(cities[j],cities[i])] for j in range(len(cities))) - \
                           mdl.sum(X_ship[(cities[i],cities[j])] for j in range(len(cities))) + \
                           X_prod[(cities[i])])

In [ ]:
X_DC

### Satisfy Production Capacity

In [ ]:
for i in range(len(cities)):
    #mdl.add_constraint(S[i]*X_DC[(cities[i])] >= mdl.sum(X_ship[(cities[i],cities[k])] for k in range(len(cities))))
    mdl.add_constraint(S[i]*X_DC[(cities[i])] >= X_prod[(cities[i])])

## Define The Objective Function

In [ ]:
mdl.minimize( \
    (mdl.sum(X_DC[(cities[i])]*F[i] for i in range(len(cities)))) + \
    (mdl.sum(X_ship[(cities[i], cities[j])]*c.iloc[i,j] for i in range(len(cities)) for j in range(len(cities)))))

In [ ]:
mdl.prettyprint()

##  Solve

In [ ]:
mdl.print_information()

In [ ]:
print(mdl.solve(url=url, key=key))

## Further Analyse of the Solution

In [ ]:
print(mdl.report())
print(mdl.solve_details)

In [ ]:
## convert your array into a dataframe
VALS = mdl.solution.get_all_values()
ARCS = pd.DataFrame(np.reshape(VALS[len(cities):(len(VALS)-len(cities))], (len(cities), len(cities))), index = cities, columns = cities)
PROD = pd.DataFrame(np.reshape(VALS[(len(VALS) - len(cities)):len(VALS)], (len(cities), 1)), index = cities, columns = ["Production"])
FACS = pd.DataFrame(np.reshape(VALS[0:len(cities)], (len(cities), 1)), index = cities, columns = ["Facility"])

In [ ]:
FACS[FACS['Facility'] == 1]

In [ ]:
ARCS

In [ ]:
PROD

In [ ]:
print(mdl.parameters.mip.prettyprint())

## Render A Visualization

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
locs = pd.DataFrame(columns = ["City", "Latitude", "Longitude"])

for i in range(len(cities)):
    location = geolocator.geocode(cities[i])
    locs.loc[i] = [cities[i], location.latitude,location.longitude]

locs.to_csv("coordinates_out.csv")

FACS_locs = pd.DataFrame(columns = ["City", "Latitude", "Longitude"])
FACS_cities = list(FACS[FACS['Facility'] == 1].index)
for i in range(len(FACS_cities)):
    location = geolocator.geocode(FACS_cities[i])
    FACS_locs.loc[i] = [FACS_cities[i], location.latitude,location.longitude]

FACS_locs.to_csv("FACScoordinates_out.csv")

## Download the code

In [ ]:
print("git clone 'https://github.com/TomKennes/Facility_Location_Problem__docplex__python.git'")
print("git clone 'https://github.com/IBMDecisionOptimization/docplex-examples.git'")